https://isabelle.in.tum.de/library/HOL/HOL/Deriv.html
https://isabelle.in.tum.de/library/HOL/HOL/Limits.html

https://guillaume.melquiond.fr/doc/14-mscs.pdf Real analysis formalization comparison

https://link.springer.com/epdf/10.1007/s10817-012-9250-9?sharing_token=6nzQoOkN5pG9Y091mWDoAPe4RwlQNchNByi7wbcMAY6JcMCFBxY4_tA5xyf86Jin_5FlGs6hw_t2rHTDUpWP0aoFz0S4zpSBogGey5bWIBYJLqkqx5UXNgn97mjVK9_8DgIBfD7TVdR533YzHpBkThPd6qBgw4bC9PP4UEs2qdc%3D harrison hol light theory of euclidean space

https://dl.acm.org/doi/10.1007/978-3-642-39634-2_21 Type classes and filters for mathematical analysis in Isabelle/HOL

filters are often ntions of quantifier. Huh


In [7]:
from kdrag.all import *
import kdrag.theories.real as real
import kdrag.theories.set as set_

x,y,z = smt.Reals("x y z")
RSet = set_.Set(smt.RealSort())
RFun = real.RFun
RSeq = real.RSeq


A,B = smt.Consts("A B", RSet)

upper_bound = kd.define("upper_bound", [A,x], kd.QForAll([y], A[y], y <= x))
lower_bound = kd.define("lower_bound", [A,x], kd.QForAll([y], A[y], y >= x))
upper_bounded = kd.define("upper_bounded", [A], smt.Exists([x], kd.QForAll([y], A[y], y <= x)))
lower_bounded = kd.define("lower_bounded", [A], smt.Exists([x], kd.QForAll([y], A[y], y >= x)))

# sequences and Functions

lub = smt.Function("lub", RSet, real.R)
lub_is_bound = kd.axiom(kd.QForAll([A], upper_bounded(A), upper_bound(A, lub(A))))
lub_is_least = kd.axiom(kd.QForAll([A, x], upper_bound(A, x), lub(A) <= x))


BSet = kd.NewType("BSet", RSet, pred=lambda x: upper_bounded(x))
#BSeq = kd.NewType("BSeq", RSeq, pred=lambda x: upper_bounded(x))

# closed under union, inter, comprehension.
# Points are bounded.
# Addition of constant is bounded




In [ ]:

def lim_tac():
def bnd_tac

In [ ]:
ZFun = kd.NewType("ZFun", RFun, pred=lambda f: converges_to_zero(f))

Could make set inherit add sub etc from underyling sort.



I should make SortDispatch auto lift liftable stuff.

of_int
of_str
of_

of_


is filter a useful concept?
https://isabelle.in.tum.de/library/HOL/HOL/Filter.html

I really want it to be generic over the sort...


The signature is generic, but the proofs are GenericProof.




In [ ]:

TRUE = kd.notation.SortDispatch("TRUE")

is_filter = kd.define("is_filter", [A], kd.QForAll([x,y], A[x] & A[y], A[x & y]))


a = smt.TypeVar("a")
Fsort = (a >> B) >> B
is_filter = smt.Function("is_filter", FSort, B)

@GenericProof
def TRUE(A):
    



In [9]:
smt.TypeVar("a")

AttributeError: module 'kdrag.smt' has no attribute 'TypeVar'

# deriv
https://isabelle.in.tum.de/library/HOL/HOL/Deriv.html

In [ ]:
real.diff_at
real.deriv
real.has_diff_at
real.

|- (Lambda(x, sin[x])) == sin

We could check if free vars arte in the expressions.

Or seal the freew vars off once 
Or we could herbandize when we enter.

Could just add the definition intyo the trace.



In [9]:
from kdrag.all import *
import kdrag.theories.real as real
x = smt.Real("x")
sin = kd.define("sin", [x], real.sin(x))
cos = kd.define("cos", [x], real.cos(x))
l = kd.Lemma(real.deriv(smt.Lambda([x], sin(x))) == smt.Lambda([x], cos(x)))
l.unfold()
l.search()
y = smt.Real("y")
l.rw(kd.prove(smt.Lambda([y], real.sin[y]) == real.sin))
#l.search()
l.rw(real.deriv_sin)
l.auto()
#l.lemmas
l.qed()

#l.qed(by=[sin.defn,cos.defn])




|- deriv(Lambda(x, sin(x))) == (Lambda(x, cos(x)))

In [15]:
from kdrag.all import *
import kdrag.theories.real as real
x = smt.Real("x")
sin = kd.define("sin", [x], real.sin(x))
cos = kd.define("cos", [x], real.cos(x))
F = smt.Const("F", real.RFun)
# explicit chain rule form.
deriv_sin = kd.axiom(smt.ForAll([F], real.deriv(smt.Lambda([x], sin(F(x)))) == smt.Lambda([x], cos(x)*real.deriv(F)[x])))

l = kd.Lemma(real.deriv(smt.Lambda([x], sin(sin(x)))) == smt.Lambda([x], cos(sin(x))*cos(x)))
print(l)
l.rw(deriv_sin)
l.lemmas

#l.rw(deriv_sin(smt.Lambda([x], sin(x))))


[] ?|- deriv(Lambda(x, sin(sin(x)))) ==
(Lambda(x, cos(sin(x))*cos(x)))


[|- Implies(ForAll(F,
                deriv(Lambda(x, sin(F[x]))) ==
                (Lambda(x, cos(x)*deriv(F)[x]))),
         deriv(Lambda(x, sin(Lambda(X!594, sin(X!594))[x]))) ==
         (Lambda(x,
                 cos(x)*deriv(Lambda(X!594, sin(X!594)))[x]))),
 |- ForAll(F,
        deriv(Lambda(x, sin(F[x]))) ==
        (Lambda(x, cos(x)*deriv(F)[x])))]

Ok, so when we substitute in, we no longer match because substitue does not work modulo simple beta reduction. The higher order thing shows up... I see.

It should be possible for deriv_tac to mostly be just applying a pile of rewrite rules.

In [ ]:
def deriv_tac(f, trace=None):
    if isinstance(f, smt.QuantifierRef) and f.is_lambda():
        vs, body = kd.utils.open_binder_unhygienieic(f)
        deriv_tac_x(vs[0], body, trace)
    elif f.eq(real.sin):
        trace.append(deriv_sin)
        return real.cos
    elif f.eq(real.cos):
        trace.append(deriv_cos)
        return -real.sin

def deriv_tac(x, e, trace=None):
    def is_app(e):
        decl = e.decl()
        if smt.is_select(e): #apply
            if e.args(1) == x:
                deriv_tac(e.args(0), trace)
        elif decl == sin:
            







## Complete Lattice


In [30]:
a = smt.DeclareTypeVar("a")
is_lattice = smt.Function("is_lattice", a, smt.BoolSort())
# or maybe this should take in a Set if it needs to take in something anway. FullSet for entire type

assert is_lattice(smt.BoolVal(True)).decl() != is_lattice(smt.IntVal(1)).decl()
is_latticeb = smt.Function("is_lattice", smt.BoolSort(), smt.BoolSort())
is_lattice(smt.BoolVal(True)).decl() == is_latticeb #hmm. Also not equal.
is_lattice(smt.BoolVal(True)).decl() == is_lattice
q = smt.Const("q", a)
is_lattice(q).decl() == is_lattice

True

In [16]:
from z3 import *
T = DeclareTypeVar("T")
List = Datatype("List")
List.declare("nil")
List.declare("cons", ("car", T), ("cdr", List))
List = List.create()
ilist = List.cons(IntVal(3), List.nil)
rlist = List.cons(RealVal(3), List.nil)
assert ilist.sort() == rlist.sort() 
assert List.car(ilist).sort() == List.car(rlist).sort() # same sort. Odd.

car(cons(3, nil))

In [ ]:
parse_smt2_file

In [10]:
from z3 import *
T = DeclareTypeVar("T")
List = Datatype("List")
List.declare("nil")
List.declare("cons", ("car", T), ("cdr", List))
List = List.create()
ilist = List.cons(IntVal(3), List.nil)
rlist = List.cons(RealVal(3), List.nil)
assert ilist.sort() == rlist.sort() # same sort. Odd.
assert simplify(List.cdr(ilist)).eq(simplify(List.cdr(rlist)))
assert not ilist.eq(rlist)
s = Solver()
s.add(List.car(ilist) == List.car(rlist))
s.check()
#assert List.car(ilist).sort() == T
#assert List.car(rlist).sort() == T
s = Solver()
ilist2 = List.cons(IntVal(4), List.nil)
#s.add(List.car(ilist) == List.car(ilist2))
#s.add(List.car(ilist) == List.car(rlist))
x = Int("x")
s.add(List.car(ilist) == x) 
s.check()

AssertionError: 

In [48]:
from z3 import *
T = DeclareTypeVar("T")
List = Datatype("List")
List.declare("nil")
List.declare("cons", ("car", T), ("cdr", List))
List = List.create()
ilist = List.cons(IntVal(3), List.nil)
rlist = List.cons(RealVal(3), List.nil)
assert ilist.sort() == rlist.sort() # same sort
assert not ilist.eq(rlist)
s = Solver()
s.add(List.car(ilist) == List.car(rlist))
s.check()
assert List.car(ilist).sort() == T
assert List.car(rlist).sort() == T
s = Solver()
ilist2 = List.cons(IntVal(4), List.nil)
#s.add(List.car(ilist) == List.car(ilist2))
#s.add(List.car(ilist) == List.car(rlist))
x = Int("x")
#s.add(List.car(ilist) == x) 
s.check()
ilist.decl() == rlist.decl()

False

In [18]:
import kdrag.theories.set as set_
L = smt.DeclareSort("L")
LSet = set_.Set(L)
a,b,c = smt.Consts("a b c", L)
le = smt.Function("le", L, L, smt.BoolSort())
le_refl = kd.axiom(kd.QForAll([a], le(a,a)))
le_trans = kd.axiom(kd.QForAll([a,b,c], le(a,b), le(b,c), le(a,c)))


kd.notation.le.register(L,le)
A = smt.Const("A", LSet)
inf = smt.Function("inf", LSet, L)
inf_le = kd.axiom(kd.QForAll([A], A[a], inf(A) <= a))
inf_is_least = kd.axiom(kd.QForAll([A, b], kd.QForAll([a], A[a], b <= a), b <= a))


f = smt.Function("f", L, L)
f_monotone = kd.axiom(kd.QForAll([a,b], le(a,b), le(f(a), f(b))))

fp = kd.define("fp", [a], f(a) == a)






In [ ]:

@GenericProof
def inf_le(L):
    LSet = set_.Set(L) 
    A = smt.Const("A", LSet)
    a = smt.Const("a", L)
    inf = smt.Function("inf", L, L)
    return kd.QForAll([A], A[a], inf(A) <= a)




#@KnuckleClass
class CompleteLattice():
    def __init__(self, L):
        self.L = L
        self.LSet = set_.Set(L)
        self.le = kd.notation.le[L]
        self.inf = inf[L]
        self.inf_le = inf_le[L] #kd.axiom(kd.QForAll([A], A[a], inf(A) <= a))
        self.inf_is_least = inf_is_least[L] #kd.axiom(kd.QForAll([A, b], kd.QForAll([a], A[a], b <= a), b <= a))

    
class CompleteLattice():
    def __init__(self, L):
        self.L = L
        self.LSet = set_.Set(L)
        self.le = kd.notation.le[L]
        self.inf = inf[L]
        self.inf_le = inf_le[L] #kd.axiom(kd.QForAll([A], A[a], inf(A) <= a))
        self.inf_is_least = inf_is_least[L] #kd.axiom(kd.QForAll([A, b], kd.QForAll([a], A[a], b <= a), b <= a))

    @classmethod
    def register(self, L, le, inf, inf_le, inf_is_least):
        # do the checks
        self.L = L
        self.LSet = set_.Set(L)
        self.le = le
        self.inf = inf
        self.inf_le = inf_le
        self.inf_is_least = inf_is_least


# we could memoize but whats the point?




That I need to make GenericProof and then rewrap is dupolication of effort.
I could make a decorator that goes through all BoolRef properties and looks up the theorems.

Maintain a mapping of 

{L : ( ,  ,  )}

Non decorated form. Use subclass mechanics intead.
dataclass decorator justy to avoid boilerplate?


In [ ]:
class SortClass():
    _registry = {}
    def lookup(self, L):
        return self.registry[L]
    @classmethod
    def __getitem__(x):

    @classmethod
    def register(self, *args, **kwargs):
        self.registry[self.L] = self(*args, **kwargs)
        return  self.



#@SortClass # typeclass?
class Lattice(SortClass):
    def __init__(self, L):
        self.L = L
        self.LSet = set_.Set(L)
        self.le = kd.notation.le[L]
        self.inf = inf[L]
        self.inf_le = inf_le[L]
        self.inf_le = Lattice.lookup(L, ForAll([], ...))
        d = Lattice[L]
        assert alpha_eq(d.inf_le, smt.ForAll([x,y,z], ...))
        Lattice[L].inf_le
    def check(self):
        assert self.inf_le == smt.ForAll([x,y,z], ...)
        assert self.inf_is_least == smt.ForAll([x,y,z], ...)
        
    

https://okmij.org/ftp/Computation/typeclass.html
https://okmij.org/ftp/Haskell/TypeClass.html
https://okmij.org/ftp/ML/canonical.html
So I could also have a registry for rules. rules could either return None or throw.
Try them one by one.


def register_rule(cls, rule):

https://lawrencecpaulson.github.io/2022/03/02/Type_classes.html
https://lawrencecpaulson.github.io/2022/03/23/Locales.html

https://link.springer.com/chapter/10.1007/978-3-642-39634-2_5  Canonical Structures for the Working Coq User

In [ ]:
class SortClass():
    def __init__(self, *L):
        for k,v in self._registry[L].items():
            setattr(self, k, v)
    @classmethod
    def get_registry(cls):
        hasattr(cls, "_registry") or setattr(cls, "_registry", {})
        return cls._registry
    @classmethod
    def lookup(cls, *L):
        return cls.get_registry()[L]
    @classmethod
    def register(cls, *L, **kwargs):
        cls.get_registry()[L] = kwargs
        cls(*L).check()

class Group(SortClass):
    def check(self, L):
        x,y,z = smt.Consts("x y z", L)
    @classmethod
    def lookup(cls, L):
        try:
            # recurse into type yada yada.
            # rules akin to  (Group A, Group B) => Group tuple(A,B)
        except :
            return cls.get_registry()[L]
        


# real vector

https://isabelle.in.tum.de/library/HOL/HOL/Real_Vector_Spaces.html

In [ ]:
from kdrag.all import *
import kdrag.theories.real as real

a = smt.DeclareTypeVar("a")
scaleR = smt.Function("scaleR", real.R, a, real.R)

def divideR(x, r):
    return scaleR(1 / r, x)


class RealVector():
    registry = {}
    def __init__(self, L):





Maybe having a higher order notion of sum isn't always good.
Just specialize the sum to named version.


In [ ]:
from kdrag.all import *
import kdrag.theories.real as real
n, m, k = smt.Ints("n m k")
f = smt.Const("f", real.RSeq)
Sum = smt.Function("Sum", real.RSeq, smt.IntSort(), smt.IntSort(), real.R)
Sum = kd.define("Sum", [f,n,m], 
                kd.cond((n == m, smt.RealVal(0)),
                        (n < m,  f[m] + Sum(f, n, m-1)),
                        (n > m, -f[m] + Sum(f, n, m+1))))


Lemma(Sum(f, n, m) == -Sum(f, m, n)


       Sum(f, n, m) ==
       If(n == m,
          0,
          If(n < m,
             f[m] + Sum(f, n, m - 1),
             If(n > m,
                -f[n] + Sum(f, n - 1, m),
                unreachable!645)))) to ForAll([f, n, m],
       Sum(f, n, m) ==
       If(n == m,
          0,
          If(n < m,
             f[m] + Sum(f, n, m - 1),
             If(n > m,
                -f[n] + Sum(f, n - 1, m),
                unreachable!650))))


LemmaError: (Implies(And,
        If(m!653 == n!652,
           0,
           If(m!653 < n!652,
              f!651[n!652] + Sum(f!651, m!653, n!652 - 1),
              If(m!653 > n!652,
                 -f!651[m!653] +
                 Sum(f!651, m!653 - 1, n!652),
                 unreachable!650))) ==
        If(m!653 == k!654,
           0,
           If(m!653 < k!654,
              f!651[k!654] + Sum(f!651, m!653, k!654 - 1),
              If(m!653 > k!654,
                 -f!651[m!653] +
                 Sum(f!651, m!653 - 1, k!654),
                 unreachable!650))) +
        If(k!654 == n!652,
           0,
           If(k!654 < n!652,
              f!651[n!652] + Sum(f!651, k!654, n!652 - 1),
              If(k!654 > n!652,
                 -f!651[k!654] +
                 Sum(f!651, k!654 - 1, n!652),
                 unreachable!650)))), 'Countermodel', [f!651 = K(Int, 0),
 k!654 = 71692,
 m!653 = 71690,
 n!652 = 71693,
 k!656 = [else -> 0],
 Sum = [(as-array, 71690, 7169

In [ ]:
l = kd.Lemma(smt.ForAll([f,n,m,k], Sum(f,m,n) == Sum(f,m,k) + Sum(f,k,n)))
_f, _m, _n, _k = l.fixes()
l.induct(_m - _n)

In [ ]:
l = kd.Lemma(smt.ForAll([f,start,size,size2], Sum(f,start, start + size1 + size2) == Sum(f,start,size1) + Sum(f,k,size2)))